# gensimを用いたTF-IDFの実装

今回は[前回の記事](https://qiita.com/kei0919/items/825901d8c9281dbbc292)の内容に対して`gensim`の`TF-IDF`を使って分析してみようと思います。
## 参考記事
TF-IDFについて　　　　　　https://mieruca-ai.com/ai/tf-idf_okapi-bm25/  
gensimのtfidfについて　　　https://qiita.com/tatsuya-miyamoto/items/f1539d86ad4980624111
## TF-IDFとは
TF-IDFが高い場合は、その文章を特徴付けるような単語となり、  
それほど重要ではない単語と言うことができると思います。  
例えば、スポーツ新聞をみており、一つの記事に注目してその記事の文章に「ホームラン」と言う単語があれば、
この記事の内容は野球についてである、とすぐに分かります。  
このような場合は野球と言う単語のTF-IDFは高い値を示す可能性が高いと思います。  
一方、ある記事の文章に選手と言う単語が入っていたとしましょう。  
スポーツ新聞なので、おそらくどの記事にも多く使われていることでしょう。  
このような場合、選手と言う単語のTF-IDFは低いと思います。  
詳しくは[こちら](https://mieruca-ai.com/ai/tf-idf_okapi-bm25/)を参考にしてください。  

今回はTF-IDFを使って記事ごとの重要な単語を抽出してみます。

では早速作業してみましょう。  
まずは必要なライプラリーを読み込みます。

In [1]:
from gensim import corpora
from gensim import models
from janome.tokenizer import Tokenizer
from pprint import pprint
import pandas as pd
import logging

ログを出力させたい場合は以下のコードを入力します。

In [2]:
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

前回の記事で作成した文春オンラインをCSVファイルとして保存していたので、そちらを利用します。

In [3]:
df = pd.read_csv('/work/data/BunshuOnline/news.csv')
doc_list = list(df.news_page_list)

記事の内容を確認してみましょう。

In [4]:
for i in range(10):
    print('%s.' % i, '〜%s〜' % df['title'][i])
    print(df['news_page_list'][i][:200], end='\n\n')

0. 〜波乱はルーキーたちの走り次第？ “怪物世代”の初陣に注目 第97回箱根駅伝予選会を読む〜
「例年なら『駅伝はトラックとは別物だから、1年生は戦力としてあまり期待しすぎないほうがいい』という話が出るんです。でも、今年はちょっと雰囲気が違いますね」 そんな風に今季の驚きを語るのは、スポーツ紙の駅伝担当記者だ。 春先から続くコロナ禍の中で、今年はここまでスポーツ界も大きな影響を受けてきた。それは学生長距離界においても同様で、春から夏にかけて大会の中止はもちろん、記録会や各校の練習にも大きな支

1. 〜スマホやタブレットじゃやっぱり不十分？ 「オンライン授業」に最適のPCとは〜
 2020年は、まさにデジタル教育の歴史にとって特記されるべき重要な年となった。まずは、小学校においてプログラミング教育が必修化された。そして、予期せぬコロナ禍の到来によって、デジタル機器を使っての学習は不可避のものとなった。 コロナで家族そろって自宅に籠っていた間、親はリモートワーク、子どもはオンライン学習に励むため、家に1台しかないPCを親子で取り合うことになった経験を持つ読者は多いだろう。 

2. 〜幼児教育「ヨコミネ式」創設者・横峯吉文氏のDVを元妻と子供が告白《法廷闘争“全面対決”へ》〜
 保育園児が跳び箱10段を跳び、倒立歩行。さらに九九算も覚え、漢字を読み書きし、3年間で平均2000冊の本を読破する――。「すべての子供が天才である」をモットーに、鹿児島から全国に広がった独創的な教育方法がある。子供の向上心や競争心に火を付け、強制することなく自分で学ぶ力を伸ばすこの教育法は「ヨコミネ式」と呼ばれている。いまでは、全国で約400もの保育園・幼稚園が、このヨコミネ式カリキュラムを導入

3. 〜BTS会社社長、韓国8位の株式富豪に…「共通点はオタクなこと」J.Y.Parkが語った素顔〜
 上場前から大騒ぎだった。 10月15日、「防弾少年団（BTS）」の所属事務所「Big Hit Entertainment（BHエンタテインメント、以下BH社）」がKOSPI（韓国取引所）に上場した。初値は公募価格の2.6倍をつけ、時価総額は8兆8169億ウォン（約8800億円同日15時時点）。韓国の3大音楽事務所の時価総額を合わせたものをはるかに超え、韓国最大手の音楽事務所に躍り出た。 創業者、

## 形態素解析
形態素解析を行います。  
`gensim`の`Tokenizer()`を用いることで簡単に実装することができます。  
記号は必要ないので、`part_of_speech`を用いて除外します。

In [5]:
t = Tokenizer()
wakati_list = []
for doc in doc_list:
    tokens = t.tokenize(doc)
    wakati = []
    for token in tokens:
        if token.part_of_speech.split(',')[0] not in ['記号']:
            wakati.append(token.base_form)
    wakati_list.append(wakati)

分かち書きと記号の除去がうまくできたか確認してみましょう。

In [6]:
for wakati in wakati_list:
    print(wakati[:10])

['例年', 'だ', '駅伝', 'は', 'トラック', 'と', 'は', '別物', 'だ', 'から']
['2020', '年', 'は', 'まさに', 'デジタル', '教育', 'の', '歴史', 'にとって', '特記']
['保育園', '児', 'が', '跳び箱', '10', '段', 'を', '跳ぶ', '倒立', '歩行']
['上場', '前', 'から', '大騒ぎ', 'だ', 'た', '10', '月', '15', '日']
['漫画', '家', 'の', '山本', 'さ', 'ほす', 'ん', 'が', '厄介', 'だ']
['三浦', '春', '馬', 'さん', 'が', '亡くなる', 'て', 'から', '3', 'カ月']
['10', '月', '1', '日', 'から', '東京', '発着', 'の', '旅行', 'も']
['自民党', '幹事', '長', 'の', '通算', '在職', '記録', 'を', '更新', '中']
['背後', 'に', '回る', 'て', '首', 'を', '左腕', 'で', '絞める', 'た']
['掛け布団', 'が', 'だんだん', '心地よい', 'なる', 'て', 'くる', 'た', '今日', 'この']


## DF-IDFを実行する前の準備を行います

次に`gensim`の`corpora`を使って単語にIDを添付していきます。  

In [7]:
dictionary = corpora.Dictionary(wakati_list)
print('==={単語: ID}===')
pprint(dictionary.token2id)

2020-10-19 06:49:28,202 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2020-10-19 06:49:28,228 : INFO : built Dictionary(2877 unique tokens: ['1', '10', '11', '12', '17']...) from 10 documents (total 13512 corpus positions)


==={単語: ID}===
{'!': 1021,
 '!!': 1822,
 ',': 2023,
 '-': 568,
 '.': 569,
 '.」': 1518,
 '.』': 1519,
 '/': 570,
 '/◆': 571,
 '000': 2024,
 '04': 1022,
 '05': 2204,
 '1': 0,
 '10': 1,
 '100': 1520,
 '11': 2,
 '12': 3,
 '13': 2205,
 '14': 2206,
 '1400': 1521,
 '15': 1522,
 '17': 4,
 '1854': 1856,
 '1939': 2207,
 '1969': 2208,
 '1971': 1023,
 '1975': 2209,
 '1981': 1024,
 '1983': 1025,
 '1997': 1026,
 '1998': 1523,
 '2': 5,
 '20': 6,
 '2000': 1027,
 '2001': 1524,
 '2005': 1525,
 '2007': 2210,
 '2009': 1526,
 '2010': 572,
 '2017': 2450,
 '2018': 573,
 '2019': 1028,
 '2020': 7,
 '2024': 8,
 '21': 2451,
 '22': 1823,
 '23': 1029,
 '24': 2025,
 '2400': 1030,
 '25': 2211,
 '251': 1857,
 '28': 9,
 '29': 10,
 '3': 11,
 '30': 1824,
 '300': 2026,
 '3000': 12,
 '3100': 1527,
 '35': 2027,
 '4': 13,
 '40': 14,
 '400': 1031,
 '42': 2028,
 '48': 2029,
 '5': 574,
 '50': 2212,
 '500': 2030,
 '5000': 15,
 '5500': 1528,
 '56': 2031,
 '58': 2213,
 '6': 1529,
 '60': 1530,
 '600': 2032,
 '61': 2033,
 '6420': 15

次に単語ごとの記事ごとの出現回数をカウントします。

In [8]:
corpus = list(map(dictionary.doc2bow, wakati_list))
print('===(単語ID, 出現回数)===')
pprint(corpus)

===(単語ID, 出現回数)===
[[(0, 10),
  (1, 5),
  (2, 1),
  (3, 1),
  (4, 1),
  (5, 5),
  (6, 1),
  (7, 1),
  (8, 1),
  (9, 1),
  (10, 1),
  (11, 2),
  (12, 1),
  (13, 1),
  (14, 1),
  (15, 2),
  (16, 3),
  (17, 3),
  (18, 2),
  (19, 1),
  (20, 1),
  (21, 5),
  (22, 1),
  (23, 1),
  (24, 1),
  (25, 11),
  (26, 2),
  (27, 1),
  (28, 1),
  (29, 1),
  (30, 2),
  (31, 1),
  (32, 3),
  (33, 21),
  (34, 5),
  (35, 1),
  (36, 1),
  (37, 3),
  (38, 3),
  (39, 1),
  (40, 1),
  (41, 3),
  (42, 10),
  (43, 65),
  (44, 8),
  (45, 1),
  (46, 1),
  (47, 2),
  (48, 1),
  (49, 12),
  (50, 5),
  (51, 2),
  (52, 1),
  (53, 4),
  (54, 1),
  (55, 1),
  (56, 1),
  (57, 1),
  (58, 2),
  (59, 1),
  (60, 26),
  (61, 1),
  (62, 1),
  (63, 1),
  (64, 1),
  (65, 3),
  (66, 5),
  (67, 1),
  (68, 1),
  (69, 3),
  (70, 1),
  (71, 1),
  (72, 2),
  (73, 5),
  (74, 37),
  (75, 1),
  (76, 2),
  (77, 12),
  (78, 2),
  (79, 43),
  (80, 1),
  (81, 3),
  (82, 1),
  (83, 1),
  (84, 1),
  (85, 44),
  (86, 36),
  (87, 6),
  (88, 4),


次に単語ごとの記事ごとの出現回数を用いていよいよTF-IDFを算出します。  
`gensim`の`models`を利用します。  
`gensim`は本当にたくさんの機能がありますね。  
計算結果の一部を表示させます。

In [16]:
test_model = models.TfidfModel(corpus)
corpus_tfidf = test_model[corpus]
print('===(単語ID, TF-IDF)===')
for doc in corpus_tfidf:
    print(doc[:4])

2020-10-19 07:00:35,502 : INFO : collecting document frequencies
2020-10-19 07:00:35,505 : INFO : PROGRESS: processing document #0
2020-10-19 07:00:35,508 : INFO : calculating IDF weights for 10 documents and 2877 features (4468 matrix non-zeros)


===(単語ID, TF-IDF)===
[(0, 0.010270560215244124), (1, 0.010876034850944512), (2, 0.011736346492935309), (3, 0.006756809998052433)]
[(0, 0.0042816896254018535), (3, 0.005633687484063879), (5, 0.008303667688079712), (7, 0.005633687484063879)]
[(0, 0.001569848428761509), (1, 0.006649579327355055), (3, 0.005163870001530652), (5, 0.00761118904775017)]
[(0, 0.004119674666568976), (1, 0.006543809340846026), (3, 0.006775642806339103), (5, 0.02496707813315839)]
[(0, 0.01276831026581211), (1, 0.013521033373868814), (7, 0.04200016584013773), (13, 0.04200016584013773)]
[(1, 0.007831949836845296), (2, 0.0422573475812842), (7, 0.024328258270175436), (11, 0.00625933452375299)]
[(0, 0.00115918318434994), (1, 0.004910079468851687), (9, 0.013246186396066643), (11, 0.0039241607229361)]
[(0, 0.0014966665107978136), (1, 0.006339594643539755), (2, 0.06841066655360874), (3, 0.009846289814745559)]
[(0, 0.0026696482016164125), (1, 0.003769373987012683), (5, 0.004314471125835394), (6, 0.007739059517798651)]
[(0,

単語IDでは分かりにくいので、単語を表示します。

In [15]:
texts_tfidf = []
for doc in corpus_tfidf:
    text_tfidf = []
    for word in doc:
        text_tfidf.append([dictionary[word[0]], word[1]])
    texts_tfidf.append(text_tfidf)
print('===[単語: TF-IDF]===')
for i in texts_tfidf:
    print(i[20:24])

===[単語: TF-IDF]===
[['U', 0.022445636964346205], ['m', 0.11222818482173101], ['あくまで', 0.022445636964346205], ['あそこ', 0.022445636964346205]]
[['しっかり', 0.02616203449412644], ['しまう', 0.002898944443406048], ['じゃ', 0.004151833844039856], ['せる', 0.005797888886812096]]
[['これ', 0.0026571889707680363], ['さらに', 0.013652529666738833], ['しまう', 0.010628755883072145], ['じゃ', 0.003805594523875085]]
[['ここ', 0.006775642806339103], ['こと', 0.008239349333137951], ['これ', 0.0034865640168190394], ['さて', 0.015732555392960215]]
[['記事', 0.006384155132906055], ['家', 0.04200016584013773], ['最近', 0.07295284301359403], ['あの', 0.09752136505414427]]
[['て', 0.053620572470353435], ['できる', 0.01792908931110876], ['です', 0.03132779934738118], ['でも', 0.0018489852575983943]]
[['その', 0.0034775495530498207], ['そもそも', 0.010081089692211678], ['て', 0.020865297318298923], ['です', 0.019640317875406748]]
[['じゃ', 0.007256376823656625], ['すぎる', 0.017102666638402184], ['せる', 0.005066636590574954], ['そう', 0.01451275364731325]]
[['しまう', 0

それではいよいよTF-IDFが上位の単語を記事ごとに表示してみましょう！  
今回はTF-IDFが0.1以上の単語のみ表示させてみます！

In [11]:
for i in range(len(texts_tfidf)):
    print('')
    print('%s.' % i, '〜%s〜' % df['title'][i])
    for text in texts_tfidf[i]:
        if text[1] > 0.13:
            print(text)


0. 〜波乱はルーキーたちの走り次第？ “怪物世代”の初陣に注目 第97回箱根駅伝予選会を読む〜
['ルーキー', 0.29179328053650067]
['予選', 0.26934764357215446]
['会', 0.2353324044944065]
['年生', 0.13467382178607723]
['校', 0.17956509571476964]
['監督', 0.13467382178607723]
['練習', 0.15711945875042344]
['走り', 0.15711945875042344]
['駅伝', 0.3591301914295393]
['高校', 0.14119944269664392]

1. 〜スマホやタブレットじゃやっぱり不十分？ 「オンライン授業」に最適のPCとは〜
['.', 0.17005322421182187]
['/', 0.2807205709669065]
['LAVIE', 0.1684323425801439]
['Office', 0.1310029331178897]
['PC', 0.561441141933813]
['子ども', 0.2092962759530115]
['学習', 0.14971763784901682]
['搭載', 0.1684323425801439]
['機能', 0.1684323425801439]

2. 〜幼児教育「ヨコミネ式」創設者・横峯吉文氏のDVを元妻と子供が告白《法廷闘争“全面対決”へ》〜
['教育', 0.15248089693189756]
['A', 0.1435114324064918]
['ミネ', 0.1715400483643072]
['ヨコ', 0.1715400483643072]
['保育園', 0.37738810640147585]
['吉文', 0.3259260918921837]
['子', 0.2744640773828915]
['子供', 0.2573100725464608]
['暴力', 0.2230020628735994]
['氏', 0.17938929050811475]
['祖母', 0.13723203869144576]
['鹿児島', 0.13723203869144576

`gensim`に便りっきりの簡単な実装でしたが、ある程度重要そうな単語のみを抽出できているのではないでしょうか？  
`gensim`は非常に便利であることが分かりました。

次回はいよいよ`word2vec`を用いてテーマを決めてクラスタリングを実装してみようかと思います！